# Framework HF

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Cara mengganti password?"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

answer = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
print("content:", answer)

# GGUF

In [ ]:
import re
from llama_cpp import Llama

# Initialize model
model = Llama(
    model_path="Qwen3-8B-Q4_K_M.gguf",
    n_ctx=1024,
    max_tokens=1,
    temperature=0.0,
    top_p=0.95,
    top_k=10,
    verbose=False,
)

# Function to generate predictions using the model and measure inference time
def generate(input_text, system_prompt="", max_length=1024):
    messages = [
        {"role": "user", "content": f"{input_text}"},
    ]
    outputs = model.create_chat_completion(messages=messages, max_tokens=max_length)
    raw_result = outputs["choices"][0]["message"]["content"]
    
    # Remove <think>...</think> block
    result = re.sub(r"<think>.*?</think>", "", raw_result, flags=re.DOTALL).strip()
    result = result.splitlines()[-1].strip()
    
    return result

#Example Usage
text = "Bagaimana cara mengganti password?"
result, inference_time = generate(text)
print(f"{result}")